# How to handle "double-texting" or concurrent runs in your graph

You might want to start a new run on a thread while the previous run still haven't finished. We call this "double-texting" or multi-tasking.

There are several strategies for handling this:
 
- `reject"`: Reject the new run.
- `interrupt`: Interrupt the current run, keeping steps completed until now, and start a new one.
- `enqueue`: Queue up the new run to start after the current run finishes.

### Reject

In [1]:
from langgraph_sdk import get_client
from langchain_core.messages import convert_to_messages
import httpx

In [2]:
client = get_client()

In [3]:
assistant = await client.assistants.create("agent")

In [4]:
thread = await client.threads.create()

In [5]:
run = await client.runs.create(
    thread["thread_id"],
    assistant["assistant_id"],
    input={"messages": [{"role": "human", "content": "whats the weather in sf?"}]}
)

In [6]:
try:
    await client.runs.create(
        thread["thread_id"],
        assistant["assistant_id"],
        input={"messages": [{"role": "human", "content": "whats the weather in nyc?"}]},
        multitask_strategy="reject",
    )
except httpx.HTTPStatusError as e:
    print("Failed to start concurrent run", e)

Failed to start concurrent run Client error '409 Conflict' for url 'http://localhost:8123/threads/a222fe40-2985-4c23-866e-2ad86850f146/runs'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/409


We can verify that the original thread finished executing:

In [7]:
# wait until the original run completes
await client.runs.join(thread["thread_id"], run["run_id"])

In [8]:
state = await client.threads.get_state(thread["thread_id"])

In [9]:
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

whats the weather in sf?
================================== Ai Message ==================================

[{'id': 'toolu_012yopy9hhFmSZaaUW4YdfiK', 'input': {'query': 'weather in san francisco'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_012yopy9hhFmSZaaUW4YdfiK)
 Call ID: toolu_012yopy9hhFmSZaaUW4YdfiK
  Args:
    query: weather in san francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1718323928, 'localtime': '2024-06-13 17:12'}, 'current': {'last_updated_epoch': 1718323200, 'last_updated': '2024-06-13 17:00', 'temp_c': 17.2, 'temp_f':

### Cancel

In [10]:
import asyncio

In [11]:
thread = await client.threads.create()

In [12]:
# the first run will be interrupted
interrupted_run = await client.runs.create(
    thread["thread_id"], assistant["assistant_id"],
    # NOTE: we add a sleep here on purpose, so that we interrupt the run before the LLM is called
    input={"messages": [{"role": "human", "content": "whats the weather in sf?"}], "sleep": 5},
)
await asyncio.sleep(1)
run = await client.runs.create(
    thread["thread_id"],
    assistant["assistant_id"],
    input={"messages": [{"role": "human", "content": "whats the weather in nyc?"}]},
    multitask_strategy="interrupt",
)

In [13]:
# wait until the second run completes
await client.runs.join(thread["thread_id"], run["run_id"])

We can see that the thread has partial data from the first run + data from the second run

In [14]:
state = await client.threads.get_state(thread["thread_id"])

In [15]:
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

whats the weather in sf?
================================ Human Message =================================

whats the weather in nyc?
================================== Ai Message ==================================

[{'id': 'toolu_01Xu1JoA4wXf9JhdvCHSv3Ub', 'input': {'query': 'weather in san francisco'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_01Xu1JoA4wXf9JhdvCHSv3Ub)
 Call ID: toolu_01Xu1JoA4wXf9JhdvCHSv3Ub
  Args:
    query: weather in san francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1718323928, 'localtime': '2024-06-13 17:12'},

Verify that the original, interrupted run was interrupted

In [16]:
(await client.runs.get(thread["thread_id"], interrupted_run["run_id"]))["status"]

'interrupted'

### Enqueue

In [17]:
thread = await client.threads.create()

In [18]:
# this run will be interrupted
first_run = await client.runs.create(
    thread["thread_id"],
    assistant["assistant_id"],
    input={"messages": [{"role": "human", "content": "whats the weather in sf?"}]}
)

In [19]:
second_run = await client.runs.create(
    thread["thread_id"],
    assistant["assistant_id"],
    input={"messages": [{"role": "human", "content": "whats the weather in nyc?"}]},
    multitask_strategy="enqueue",
)

Verify that the thread has data from both runs

In [20]:
# wait until the second run completes
await client.runs.join(thread["thread_id"], second_run["run_id"])

In [21]:
state = await client.threads.get_state(thread["thread_id"])

In [22]:
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

whats the weather in sf?
================================== Ai Message ==================================

[{'id': 'toolu_01GPfkeKvPUKUFvF6ZZBnwCd', 'input': {'query': 'weather in san francisco'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_01GPfkeKvPUKUFvF6ZZBnwCd)
 Call ID: toolu_01GPfkeKvPUKUFvF6ZZBnwCd
  Args:
    query: weather in san francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1718323928, 'localtime': '2024-06-13 17:12'}, 'current': {'last_updated_epoch': 1718323200, 'last_updated': '2024-06-13 17:00', 'temp_c': 17.2, 'temp_f':